## 학습 목표 

- 머신러닝을 위해서, numpy와 pandas의 사용법을 다시 연습합니다.

- numpy를 사용하여 여러가지 벡터 연산을 연습하고, 실제 데이터 유사도를 계산해봅시다.

### 1. 라이브러리 연습 예제 

[ 필수 RECAP ]

> Numpy : broadcasting, universal function

> Pandas : fancy indexing, categorical variable encoding

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# data를 불러와 봅니다.
base_path='/content/drive/MyDrive/Colab Notebooks/data/prudential/'
data = pd.read_csv(base_path +'train.csv')

In [29]:
data.head(5)

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,...,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,...,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,...,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,...,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,...,0,0,0,0,0,0,0,0,0,8


### 2. 유사도 측정 예제

#### EDA & Preprocessing

- 퀵하게 데이터를 파악하고, 유사도 측정 예시에 알맞게 가공해봅시다.

In [30]:
## EDA

# ins_age 0.5이상 
# data.loc[data.Ins_Age >= 0.5 , 'Ins_Age']

# medical keyword라고 시작하는 모든 컬럼 뽑아보자
data.columns[data.columns.str.startswith("Medical_Keyword")]

# 위의 여러 조건 합치기
data.loc[data.Ins_Age >= 0.5, data.columns.str.startswith("Medical_Keyword") ]

,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,...,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59369,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
data = data.drop(columns="Id")

In [31]:
## Id column을 drop하고, product_info_2 column을 숫자로 변경
# 1. id drop
data = data.drop(columns="Id")

# 2. Product_info_2를 숫자로 변경. Ordinal Encoding, One-Hot Encoding
"""
매우 싫음 싫음 보통 좋음 매우 좋음 -> Ordinal

남자 여자 trans 중성 응답x -> one-hot
"""

# 2-1. Ordinal Encoding(pandas)
data.Product_Info_2 =  pd.factorize(data.Product_Info_2)[0]

data

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,0,10,0.076923,2,1,1,0.641791,0.581818,...,0,0,0,0,0,0,0,0,0,8
1,5,1,1,26,0.076923,2,3,1,0.059701,0.600000,...,0,0,0,0,0,0,0,0,0,4
2,6,1,2,26,0.076923,2,3,1,0.029851,0.745455,...,0,0,0,0,0,0,0,0,0,8
3,7,1,3,10,0.487179,2,3,1,0.164179,0.672727,...,0,0,0,0,0,0,0,0,0,8
4,8,1,4,26,0.230769,2,3,1,0.417910,0.654545,...,0,0,0,0,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,79142,1,7,10,0.230769,2,3,1,0.074627,0.709091,...,0,0,0,0,0,0,0,0,0,4
59377,79143,1,0,26,0.230769,2,3,1,0.432836,0.800000,...,0,0,0,0,0,0,0,0,0,7
59378,79144,1,2,26,0.076923,2,3,1,0.104478,0.745455,...,0,0,0,0,0,0,0,0,0,8
59379,79145,1,4,10,0.230769,2,3,1,0.507463,0.690909,...,1,0,0,0,0,0,0,0,0,8


#### 유사도(Proximity)

1. Euclidean distance

2. Cosine Similarity

In [32]:
v1= np.array([1,2,3])
v2= np.array([0,0,0])

In [33]:
# numpy broadcasting
v1 +v2

v1 *v2 # 내적은 아니고 그냥 원소끼리 곱함 elementwise multiplication


1/v1 # 뒤집힌다

array([1.        , 0.5       , 0.33333333])

In [34]:
# np로 유클리드 거리계산 만들기
np.sqrt(np.sum((v1-v2)**2))

np.linalg.norm(v1-v2) # 벡터의 크기 norm의 디폴트값은 L2  인자는 벡터하나가 들어가야함


3.7416573867739413

In [35]:
f = lambda x,y : np.sqrt(np.sum(np.square(x-y)))
f(v1,v2)

3.7416573867739413

In [84]:
def euclidean_distance(x,y):
    return np.linalg.norm(x-y)


In [ ]:
np.sqrt(np.sum(np.square(x-y)))

In [38]:
def cosine_similarity(x,y):
    # 1. 자 : x o y
    dot = x @ y

    # 2. 분모 : |x|, |y|
    norm = np.linalg.norm(x) * np.linalg.norm(y)  + 1e-12  # (epsilon) 의미 없는 작은 수라서 0으로 나눠지는 것을 방지
    return dot/norm

### 6번 고객과 가장 가까운 고객 찾기!!

In [61]:
target= data.loc[2]
target

Product_Info_1        1.000000
Product_Info_2        2.000000
Product_Info_3       26.000000
Product_Info_4        0.076923
Product_Info_5        2.000000
Product_Info_6        3.000000
Product_Info_7        1.000000
Ins_Age               0.029851
Ht                    0.745455
Wt                    0.288703
BMI                   0.428780
Employment_Info_1     0.030000
Employment_Info_2     9.000000
Employment_Info_3     1.000000
Name: 2, dtype: float64

In [57]:
data =data.iloc[:, :14]

In [86]:
distances = []
for  idx,row in data.iterrows():
    distances.append(euclidean_distance(target,row))

In [87]:
np.argmin(distances)

2

In [88]:
distances[2] = 99999


In [89]:
np.argmin(distances)

18154